In [0]:
#!pip install --upgrade tables
#!pip install eli5

In [0]:
import pandas as pd
import numpy as np

from sklearn.dummy import DummyRegressor
from sklearn.tree import DecisionTreeRegressor

from sklearn.metrics import mean_absolute_error as mae
from sklearn.model_selection import cross_val_score

import eli5
from eli5.sklearn import PermutationImportance

# READ DATA

In [13]:
cd '/content/drive/My Drive/Colab Notebooks/dw_matrix/matrix_two/dw_matrix_carprice'

/content/drive/My Drive/Colab Notebooks/dw_matrix/matrix_two/dw_matrix_carprice


In [16]:
df = pd.read_hdf('data/car.h5')
df.shape

(106494, 155)

In [17]:
df.columns

Index(['breadcrumb', 'created_at', 'price_currency', 'price_details',
       'price_value', 'seller_address', 'seller_name', 'seller_type',
       'feature_czujniki-parkowania-przednie',
       'feature_poduszka-powietrzna-chroniąca-kolana',
       ...
       'param_pearl', 'param_stan', 'param_wersja', 'param_emisja-co2',
       'param_body-type', 'param_matowy', 'param_bezwypadkowy',
       'param_akryl-(niemetalizowany)', 'param_monthly-payment-value',
       'car_id'],
      dtype='object', length=155)

##Dummmy Model

In [19]:
feats  = ['car_id']
X = df[feats].values
y = df['price_value'].values

model = DummyRegressor()
model.fit(X, y)
y_pred = model.predict(X)
mae(y, y_pred)

39465.934630440985

In [20]:
df['price_currency'].value_counts()

PLN    106290
EUR       204
Name: price_currency, dtype: int64

In [0]:
df = df[df['price_currency'] != 'EUR']

In [22]:
df['price_currency'].value_counts()

PLN    106290
Name: price_currency, dtype: int64

##FEATRUES

In [0]:
SUFFIX_CAT = '__cat'
for feat in df.columns:
  if isinstance(df[feat][0], list): continue
  factorized_values = df[feat].factorize()[0]
  if SUFFIX_CAT in feat:
    df[feat] = factorized_values
  else:
    df[feat + SUFFIX_CAT] = df[feat].factorize()[0]

In [30]:
cat_feats = [ x for x in df.columns if SUFFIX_CAT in x]
cat_feats = [ x for x in cat_feats if 'price' not in x]
len(cat_feats)

151

In [0]:
X = df[cat_feats].values
y = df['price_value'].values

In [32]:
model = DecisionTreeRegressor(max_depth=5)
scores = cross_val_score(model, X, y, cv=3, scoring='neg_mean_absolute_error')
np.mean(scores)

-19566.588937368324

In [35]:
m = DecisionTreeRegressor(max_depth=5)
m.fit(X,  y)

importances = PermutationImportance(m, random_state=2020).fit(X, y)
eli5.show_weights(importances, feature_names=cat_feats)

Weight,Feature
0.2564 ± 0.0064,param_napęd__cat
0.2014 ± 0.0063,param_faktura-vat__cat
0.1965 ± 0.0050,param_stan__cat
0.1429 ± 0.0042,param_rok-produkcji__cat
0.0630 ± 0.0037,param_moc__cat
0.0422 ± 0.0015,param_skrzynia-biegów__cat
0.0418 ± 0.0007,feature_kamera-cofania__cat
0.0301 ± 0.0059,param_marka-pojazdu__cat
0.0193 ± 0.0030,param_pojemność-skokowa__cat
0.0163 ± 0.0005,feature_bluetooth__cat
